In [ ]:
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, ZoomControl, Marker
import ipywidgets as widgets
from ipyflex import FlexLayout 
import json 
import os
import json
import random
import requests
import Functions as fct

In [ ]:
if not os.path.exists('countries.geojson'):
    url = 'https://github.com/datasets/geo-countries/blob/master/data/countries.geojson'
    r = requests.get(url)
    with open('countries_copy', 'w') as f:
        f.write(r.content.decode("utf-8"))
   
with open('countries.geojson', 'r') as f:
    data = json.load(f)

def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

m = Map(center=(50.6252978589571, 0.34580993652344), zoom=3)

geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)
m.add(geo_json)

m

In [ ]:
from ipyleaflet import Map
from geopy.geocoders import Nominatim
import asyncio
from ipywidgets import Output, VBox

box = VBox()


def handle_click(**kwargs):
    location = marker.location
    country = get_country_from_coordinates(location)
    #print(f"Clicked location: {location}, Country: {country}")
    x=fct.Countries_Abrreviation(country)

    asyncio.create_task(fct.Carbon_Intensity_per_day(x, box))


def get_country_from_coordinates(coordinates):
    geolocator = Nominatim(user_agent="geo_locator")
    location = geolocator.reverse(coordinates, language="en")
    address = location.address
    country = address.split(",")[-1].strip()
    return country

center = (52.204793, 360.121558)
m = Map(center=center, zoom=2)  
marker = Marker(location=center, draggable=True)
m.add_layer(marker)
marker.on_click(handle_click)

box.children = [m]
display(box)

In [ ]:
await fct.Carbon_Intensity_per_day('FR', box)